# Introduction to Probability and Statistics
Sa notebook na ito, maglalaro tayo gamit ang ilan sa mga konseptong natalakay na natin. Maraming konsepto mula sa probabilidad at estadistika ang mahusay na naipapakita sa mga pangunahing library para sa pagpoproseso ng datos sa Python, tulad ng `numpy` at `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Random Variables and Distributions
Magsimula tayo sa pagkuha ng 30 na halimbawang halaga mula sa pantay-pantay na distribusyon mula 0 hanggang 9. Kakalkulahin din natin ang mean at variance.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Upang tantiyahin nang biswal kung ilan ang iba't ibang halaga sa sample, maaari nating iguhit ang **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Pagsusuri ng Totoong Data

Mahalaga ang mean at variance kapag sinusuri ang totoong datos mula sa mundo. I-load natin ang datos tungkol sa mga manlalaro ng baseball mula sa [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Gumagamit tayo ng isang pakete na tinatawag na [**Pandas**](https://pandas.pydata.org/) dito para sa pagsusuri ng datos. Pag-uusapan pa natin nang mas detalyado ang tungkol sa Pandas at ang pagtrabaho sa datos gamit ang Python sa susunod na bahagi ng kursong ito.

I-compute natin ang mga karaniwang halaga para sa edad, taas, at timbang:


In [ ]:
df[['Age','Height','Weight']].mean()

Ngayon, magtuon tayo sa taas, at kwentahin ang standard deviation at variance:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Bilang karagdagan sa mean, makatuwiran rin na tingnan ang median na halaga at mga quartile. Maaari silang maipakita gamit ang isang **box plot**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Maaari rin tayong gumawa ng mga box plot ng mga subset ng ating dataset, halimbawa, na pinag-grupo ayon sa papel ng manlalaro.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Nota**: Ipinapahiwatig ng diagram na ito, na sa karaniwan, mas mataas ang tangkad ng mga first basemen kaysa sa mga second basemen. Sa susunod ay malalaman natin kung paano natin masusubukan ang hypothesis na ito nang mas pormal, at kung paano ipapakita na ang ating datos ay estadistikang mahalaga upang patunayan iyon.  

Ang edad, tangkad, at timbang ay lahat ng mga tuloy-tuloy na random na variable. Ano sa tingin mo ang kanilang distribusyon? Isang magandang paraan upang malaman ay ang iguhit ang histogram ng mga halaga: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normal Distribution

Gumawa tayo ng isang artipisyal na sample ng mga timbang na sumusunod sa normal na distribusyon na may parehong mean at variance tulad ng sa totoong data natin:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Dahil karamihan sa mga halaga sa totoong buhay ay karaniwang naipapamahagi, hindi natin dapat gamitin ang isang pantay-pantay na random number generator upang makabuo ng sample data. Ganito ang nangyayari kung susubukan nating gumawa ng mga timbang gamit ang isang pantay na pamamahagi (na ginawa gamit ang `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Confidence Intervals

Ngayon ay kalkulahin natin ang mga confidence interval para sa mga timbang at taas ng mga manlalaro ng baseball. Gagamitin natin ang code [mula sa talakayan sa stackoverflow na ito](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Pagsusuri ng Hipotesis

Tuklasin natin ang iba't ibang mga papel sa aming dataset ng mga manlalaro ng baseball:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Subukan natin ang hipotesis na mas matangkad ang mga First Basemen kaysa mga Second Basemen. Ang pinakasimpleng paraan upang gawin ito ay subukan ang mga confidence intervals:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Makikita natin na ang mga interval ay hindi nag-o-overlap.

Isang mas estadistikang tamang paraan upang patunayan ang hipotesis ay ang paggamit ng **Student t-test**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Ang dalawang halaga na ibinabalik ng function na `ttest_ind` ay:
* Ang p-value ay maaaring ituring bilang ang posibilidad na ang dalawang distribusyon ay may parehong mean. Sa aming kaso, ito ay napakababa, na nangangahulugang may malakas na ebidensya na sumusuporta na ang mga first basemen ay mas matangkad.
* Ang t-value ay ang panggitnang halaga ng na-normalize na pagkakaiba ng mean na ginagamit sa t-test, at ito ay ihinahambing sa isang threshold value para sa isang ibinigay na halaga ng kumpiyansa.


## Pagsasalarawan ng Normal na Distribusyon gamit ang Central Limit Theorem

Ang pseudo-random generator sa Python ay idinisenyo upang magbigay sa atin ng pantay-pantay na distribusyon. Kung nais nating gumawa ng generator para sa normal na distribusyon, maaari nating gamitin ang central limit theorem. Upang makakuha ng isang normal na distributed na halaga, kukunin lang natin ang mean ng isang sample na ginawa gamit ang uniform na distribusyon.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Correlation at Evil Baseball Corp

Pinapahintulutan tayo ng correlation na makita ang mga relasyon sa pagitan ng mga sunud-sunod na datos. Sa aming halimbawa, magpalagay tayo na may isang masamang kumpanya ng baseball na nagbabayad sa mga manlalaro nito ayon sa kanilang taas - kung mas matangkad ang manlalaro, mas malaki ang perang matatanggap niya. Ipagpalagay na may base salary na $1000, at karagdagang bonus mula $0 hanggang $100, depende sa taas. Kukuhanin natin ang mga totoong manlalaro mula sa MLB, at kakalkulahin ang kanilang mga imahinaryong sweldo:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Ngayon ay kwentahin natin ang covariance at correlation ng mga sequences na iyon. Ang `np.cov` ay magbibigay sa atin ng tinatawag na **covariance matrix**, na isang pagpapalawak ng covariance sa maraming variable. Ang elementong $M_{ij}$ ng covariance matrix na $M$ ay isang correlation sa pagitan ng mga input variable na $X_i$ at $X_j$, at ang mga diagonal na halaga na $M_{ii}$ ay ang variance ng $X_{i}$. Katulad nito, ang `np.corrcoef` ay magbibigay sa atin ng **correlation matrix**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Ang isang correlation na katumbas ng 1 ay nangangahulugan na mayroong malakas na **linear na ugnayan** sa pagitan ng dalawang variable. Makikita natin nang biswal ang linear na ugnayan sa pamamagitan ng pag-plot ng isang halaga laban sa isa pa:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Tingnan natin kung ano ang mangyayari kung ang relasyon ay hindi linear. Ipagpalagay na ang aming korporasyon ay nagpasya na itago ang maliwanag na linear na ugnayan sa pagitan ng mga taas at suweldo, at nagpakilala ng ilang non-linearity sa pormula, tulad ng `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Sa kasong ito, medyo mas maliit ang korelasyon, pero ito ay nananatiling medyo mataas. Ngayon, para gawing mas hindi halata ang relasyon, maaaring gusto nating magdagdag ng dagdag na randomness sa pamamagitan ng pagdagdag ng isang random na variable sa suweldo. Tingnan natin kung ano ang mangyayari:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Mahuhulaan mo ba kung bakit ang mga tuldok ay nakahanay sa mga patayong linya tulad nito?

Napansin natin ang ugnayan sa pagitan ng isang artipisyal na nilikhang konsepto tulad ng sahod at ang naobserbahang variable na *taas*. Tingnan din natin kung nagkakaugnay ang dalawang naobserbahang variable, tulad ng taas at bigat:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Sa kasamaang palad, wala kaming nakuha na mga resulta - puro kakaibang mga `nan` na halaga lamang. Ito ay dahil ang ilan sa mga halaga sa aming serye ay hindi natukoy, na kinakatawan bilang `nan`, na nagdudulot na ang resulta ng operasyon ay hindi rin natukoy. Sa pagtingin sa matrix makikita natin na ang `Weight` ang problemadong column, dahil ang self-correlation sa pagitan ng mga halaga ng `Height` ay na-kompyut.

> Ipinapakita ng halimbawa na ito ang kahalagahan ng **paghahanda ng data** at **paglilinis**. Kung walang tamang data hindi tayo makakakompyut ng anuman.

Gamitin natin ang `fillna` na pamamaraan upang punan ang nawawalang mga halaga, at kompyutin ang correlation:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Talaga namang may ugnayan, ngunit hindi kasing lakas tulad ng sa aming artipisyal na halimbawa. Sa katunayan, kung titingnan natin ang scatter plot ng isang halaga laban sa isa pa, ang relasyon ay magiging hindi gaanong halata:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Konklusyon

Sa notebook na ito natutunan natin kung paano magsagawa ng mga pangunahing operasyon sa data upang makalkula ang mga estadistikal na punsyon. Ngayon ay alam na natin kung paano gamitin ang isang matibay na kagamitan ng matematika at estadistika upang patunayan ang ilang mga hipotesis, at kung paano kalkulahin ang mga confidence interval para sa mga arbitraryong variable batay sa isang sample ng data.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Paunawa**:
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagamat sinisikap naming maging tumpak, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o kamalian. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na pinagmumulan ng katotohanan. Para sa mga mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring mangyari mula sa paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
